In [2]:
%load_ext autoreload
%autoreload 2
%reload_ext autoreload

from src import PublicAnalysis,PublicPredictor

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [3]:
file_path = "data/meter/apt_all.xlsx"
pa = PublicAnalysis(file_path)

In [4]:
pa.set_init()

### Report 20220714
- 최신 버전의 가격표로 바꾼 후로 오차가 커지는 현상 발생 분석 (1000 케이스 테스트에서 확인)
- PublicAnalysis.bill_table.get 사용하여 문제점 살펴보기

In [5]:
month = 1
pa.select_month(1)

pp = PublicPredictor(
        pa.meter_month,
        pa.month,
        public_percentage=30
    ).min_chk()

pp.predict

[10190608.999999998, 10188450.0, 2158.9999999981374]

In [9]:
import random as ran
import pandas as pd
import numpy as np

pp_test = list()
pp_record = list()

for _ in range(0,1000):
    n = ran.randrange(50, len(pa.meter_year.columns) + 1)
    _m = pa.meter_year.sample(n=n, axis=1)
    _month = ran.randrange(1, 13)
    
    month_df = pd.DataFrame(_m.loc[_month])
    month_df.reset_index(inplace=True)
    month_df.columns = ['name', 'usage (kWh)']
    
    PUBLIC_PERCENTAGE = ran.randrange(10, 60)
    households_kWh = month_df['usage (kWh)'].sum()
    APT = round((households_kWh * 100) / (100 - PUBLIC_PERCENTAGE))
    
    pp = PublicPredictor(
                    month_df,
                    _month,
                    kwh=APT
                ).min_chk()
    pp_result = pp.predict
    
    pp_test.append(pp_result[2])
    pp_record.append(pp)
    
pp_test = np.array(pp_test)

In [10]:
err_idx = pp_test.argmax()
pp = pp_record[err_idx]
print(err_idx, pp_test[err_idx])

351 4056.5
